In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
# 스파크 환경 설정
conf = SparkConf().setMaster('local').setAppName("saprk_sql_study")

In [3]:
# spark Context 생성
sc = SparkContext(conf = conf)

22/03/30 23:13:02 WARN Utils: Your hostname, devkhk-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.30.1.27 instead (on interface en0)
22/03/30 23:13:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/30 23:13:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.port', '49769'),
 ('spark.driver.host', '172.30.1.27'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1648649583948'),
 ('spark.app.startTime', '1648649582844'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'saprk_sql_study'),
 ('spark.ui.showConsoleProgress', 'true')]

In [5]:
ticker = sc.parallelize([
    (1, ("Google", "GOOGL", "USA")),
    (2, ("Netflix", "NFLX", "USA")),
    (3, ("Amazon", "AMZN", "USA")),
    (4, ("Tesla", "TSLA", "USA")),
    (5, ("Samsung", "005930", "Korea")),
    (6, ("Kakao", "035720", "Korea")),
])

In [6]:
prices = sc.parallelize([
    (1, (2984, "USA")),
    (2, (645, "USA")),
    (3, (3518, "USA")),
    (4, (1222, "USA")),
    (5, (70600, "USA")),
    (6, (125000, "USA")),
])

# 문제. 2000달러 이상의 미국 주식만 가져오기

- 가능한 방법은 몇가지 일까

In [17]:
# case1
# inner join 이후 filter

ticker_price = ticker.join(prices)
ticker_price.filter(lambda x : x[1][0][2] == "USA" and x[1][1][0] >= 2000).collect()

[(1, (('Google', 'GOOGL', 'USA'), (2984, 'USA'))),
 (3, (('Amazon', 'AMZN', 'USA'), (3518, 'USA')))]

In [18]:
# case2
# filter 이후 join

filter_ticker = ticker.filter(lambda x : x[1][2] == "USA")
filter_prices = prices.filter(lambda x : x[1][0] >= 2000)

filter_ticker.join(filter_prices).collect()


[(1, (('Google', 'GOOGL', 'USA'), (2984, 'USA'))),
 (3, (('Amazon', 'AMZN', 'USA'), (3518, 'USA')))]

22/03/31 00:52:10 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1054869 ms exceeds timeout 120000 ms
22/03/31 00:52:10 WARN SparkContext: Killing executors is not supported by current scheduler.


퍼포먼스는 case 2가 더 좋다.
셔플링을 최소화 할 수 있기 때문

매번 이런 고민을 해야한다면 개발자에 따라 코드에 따라 매번 성능이 들쑥 날쑥할것이다.

하지만 구조화된 데이터들은 스파크에서 자동으로 최적화가 가능하게 해준다.

## 구조화된 데이터의 종류
### Unstructure  : free form
- 로그파일
- 이미지

### Semi Structured : 행과 열
- csv
- json
- xml

### Structured : 행과 열 + 데이터 타입 (스키마)
- 데이터베이스


### Spark SQL
Structured Data 에서는 데이터의 구조를 이미 알고 있으므로 어떤 테스를 수행할 것인지 정의만 하면된다.
최적화를 자동으로 할 수 있음

### Spark SQL 사용 목적
- 스파크 프로그래밍 내부에서 관계형 처리를 하기 위해
- 스키마의 정보를 이용해 자동으로 최적화를 하기 위해
- 외부 데이터셋을 사용하기 쉽게 하기 위해